# A modular GCD algorithm: Overview of algorithm

The individual steps are the same as in K. Weber et al. / Journal of Algorithms 54 (2005) 152–167. Given are two integers $a, b \in \mathbb{Z}^*$. This notebook serves as an overview of the algorithm.

In [ ]:
a = 7 * 5 * 7 * 7 * 3
b = 7 * 5 * 11 * 5
a, b

We write $U$ and $V$ for $a$ and $b$ with $U \geq V$.

In [2]:
if a >= b:
    U = a
    V = b
else:
    U = b
    V = a

U, V

(5145, 1925)

## MGCD1: [Find suitable moduli]

$$
\begin{align}
1 \quad & n = \lfloor \text{lg} U \rfloor + 1 \\
2 \quad & w = \text{ an integer satisfying }\pi(2^w) - \pi(2^{w-1}) \leq \text{max}\lbrace \lceil 2^{w/2}+n \rceil, 9 \rbrace \\
3 \quad & Q = \text{ the set of }\lceil 2^{w/2} + n \rceil\text{ largest primes less than }2 w
\end{align}
$$

In [3]:
n = floor(log(U, 2)) + 1
n

13

In [4]:
def pi(x):
    P = Primes()
    i = 0
    while P.unrank(i) <= x:
        i += 1

    return i

In [5]:
w = 1
while pi(2^w) - pi(2^(w-1)) < max(ceil(2^(w/2) + n), 9):
    w += 1

w

9

In [6]:
def getPrimes_mod(number, bound):
    primes = set()
    P = Primes()
    i = 0
    while P.unrank(i) < bound:
        i += 1

    while len(primes) < number:
        assert(i > 0)
        i -= 1
        primes.add(P.unrank(i))

    return primes

In [7]:
Q = getPrimes_mod(ceil(2^(w/2) + n), 2^w)
Q

{293,
 307,
 311,
 313,
 317,
 331,
 337,
 347,
 349,
 353,
 359,
 367,
 373,
 379,
 383,
 389,
 397,
 401,
 409,
 419,
 421,
 431,
 433,
 439,
 443,
 449,
 457,
 461,
 463,
 467,
 479,
 487,
 491,
 499,
 503,
 509}

## MGCD2: [Convert to modular representation]

$$
\begin{align}
1 \quad & \text{For all }q ∈ \mathcal{Q}\text{ do }[u_q, v_q ] = [U\text{ mod } q, V\text{ mod }q]
\end{align}
$$

In [8]:
def mod(b, m):
    y = b % m
    if y > m / 2:
        y -= m

    return y

In [9]:
u, v = {}, {}
for q in Q:
    u[q], v[q] = mod(U, q), mod(V, q)

u, v

({293: -129,
  307: -74,
  311: -142,
  313: 137,
  317: 73,
  331: -151,
  337: 90,
  347: -60,
  349: -90,
  353: -150,
  359: 119,
  367: 7,
  373: -77,
  379: -161,
  383: 166,
  389: 88,
  397: -16,
  401: -68,
  409: -172,
  419: 117,
  421: 93,
  431: -27,
  433: -51,
  439: -123,
  443: -171,
  449: 206,
  457: 118,
  461: 74,
  463: 52,
  467: 8,
  479: -124,
  487: -212,
  491: 235,
  499: 155,
  503: 115,
  509: 55},
 {293: -126,
  307: 83,
  311: 59,
  313: 47,
  317: 23,
  331: -61,
  337: -97,
  347: -157,
  349: -169,
  353: 160,
  359: 130,
  367: 90,
  373: 60,
  379: 30,
  383: 10,
  389: -20,
  397: -60,
  401: -80,
  409: -120,
  419: -170,
  421: -180,
  431: 201,
  433: 193,
  439: 169,
  443: 153,
  449: 129,
  457: 97,
  461: 81,
  463: 73,
  467: 57,
  479: 9,
  487: -23,
  491: -39,
  499: -71,
  503: -87,
  509: -111})

## MGCD3: [Reduction loop]

$$
\begin{align}
1 \quad & \text{Repeat} \\
2 \quad & \quad \mathcal{P} = \lbrace q ∈ \mathcal{Q} | V\text{ mod }q \neq 0 \rbrace \\
3 \quad & \quad p =\text{ an element of }\mathcal{P}\text{ for which }|u_p / v_p\text{ mod }p|\text{ is minimal.} \\
4 \quad & \quad b = u_p / v_p\text{ mod }p \\
5 \quad & \quad Q = Q - \lbrace p \rbrace \\
6 \quad & \quad \text{For all }q ∈ \mathcal{Q}\text{ do }[u_q, v_q] = [v_q, (u_q - b v_q )/p\text{ mod }q] \\
7 \quad & \text{Until }∀q ∈ \mathcal{Q}, v_q = 0
\end{align}
$$

In [16]:
P = set()
for q in Q:
    if mod(V, q) != 0:
        P.add(q)

P

{293,
 307,
 311,
 313,
 317,
 331,
 337,
 347,
 349,
 353,
 359,
 373,
 379,
 383,
 389,
 397,
 401,
 409,
 419,
 421,
 431,
 433,
 439,
 443,
 449,
 457,
 461,
 463,
 467,
 479,
 487,
 491,
 499,
 503,
 509}

In [17]:
def findPrime_mod(P, u, v):
    minimum = None
    current_p = None
    for p in P:
        if v[p] != 0:
            m = abs(mod(u[p] / v[p], p))
            if minimum == None or m < minimum:
                minimum = m
                current_p = p
    
    return current_p        

In [18]:
p = findPrime_mod(P, u, v)
p

491

In [19]:
b = mod((u[p] / v[p]), p)
b

55

In [20]:
Q.remove(p)
Q

{293,
 307,
 311,
 313,
 317,
 331,
 337,
 347,
 349,
 353,
 359,
 373,
 379,
 383,
 389,
 397,
 401,
 409,
 419,
 421,
 431,
 433,
 439,
 443,
 449,
 457,
 461,
 463,
 467,
 479,
 487,
 499,
 503,
 509}

In [21]:
del u[p], v[p]
for q in Q:
    u[q], v[q] = v[q], mod((u[q] - b * v[q]) / p, q)

u, v

({293: 35,
  307: 35,
  311: 35,
  313: 35,
  317: 35,
  331: 35,
  337: 35,
  347: 35,
  349: 35,
  353: 35,
  359: 35,
  373: 35,
  379: 35,
  383: 35,
  389: 35,
  397: 35,
  401: 35,
  409: 35,
  419: 35,
  421: 35,
  431: 35,
  433: 35,
  439: 35,
  443: 35,
  449: 35,
  457: 35,
  461: 35,
  463: 35,
  467: 35,
  479: 35,
  487: 35,
  499: 35,
  503: 35,
  509: 35},
 {293: 0,
  307: 0,
  311: 0,
  313: 0,
  317: 0,
  331: 0,
  337: 0,
  347: 0,
  349: 0,
  353: 0,
  359: 0,
  373: 0,
  379: 0,
  383: 0,
  389: 0,
  397: 0,
  401: 0,
  409: 0,
  419: 0,
  421: 0,
  431: 0,
  433: 0,
  439: 0,
  443: 0,
  449: 0,
  457: 0,
  461: 0,
  463: 0,
  467: 0,
  479: 0,
  487: 0,
  499: 0,
  503: 0,
  509: 0})

We have to repeat the reduction loop until $∀q ∈ \mathcal{Q}, v_q = 0$.

## MGCD4: [Return standard representation]

$$
\begin{align}
1 \quad & G = 0 \\
2 \quad & \text{Repeat} \\
3 \quad & \quad p =\text{ an element of }\mathcal{Q} \\
4 \quad & \quad \mathcal{Q} = \mathcal{Q} - \lbrace p\rbrace \\
5 \quad & \quad G = u_p + p G \\
6 \quad & \quad \text{For all }q ∈ \mathcal{Q}\text{ do }u_q = (u_q - u_p) / p\text{ mod }q \\
7 \quad & \text{Until }∀q ∈ \mathcal{Q}, u_q = 0 \\
8 \quad & \text{Return }|G|
\end{align}
$$

In [22]:
G = 0

Now we begin the backconversion loop.

In [23]:
p = Q.pop()
p

311

In [24]:
Q

{293,
 307,
 313,
 317,
 331,
 337,
 347,
 349,
 353,
 359,
 373,
 379,
 383,
 389,
 397,
 401,
 409,
 419,
 421,
 431,
 433,
 439,
 443,
 449,
 457,
 461,
 463,
 467,
 479,
 487,
 499,
 503,
 509}

In [25]:
G = u[p] + p * G
G

35

In [26]:
for q in Q:
    u[q] = mod((u[q] - u[p]) / p, q)

del u[p]
u

{293: 0,
 307: 0,
 313: 0,
 317: 0,
 331: 0,
 337: 0,
 347: 0,
 349: 0,
 353: 0,
 359: 0,
 373: 0,
 379: 0,
 383: 0,
 389: 0,
 397: 0,
 401: 0,
 409: 0,
 419: 0,
 421: 0,
 431: 0,
 433: 0,
 439: 0,
 443: 0,
 449: 0,
 457: 0,
 461: 0,
 463: 0,
 467: 0,
 479: 0,
 487: 0,
 499: 0,
 503: 0,
 509: 0}

We have to repeat the backconversion loop until $∀q ∈ \mathcal{Q}, u_q = 0$.

In [27]:
abs(G)

35

This is the final result.